# Activation Tuning

In [41]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import kerastuner as kt # for tuning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense 
from tensorflow.keras import regularizers

In [42]:
df = pd.read_csv('diabetes.csv')
X = df.drop(columns=['Outcome'])
y = df['Outcome']
X_train, X_test, y_train, y_test = train_test_split(
     X, y, test_size=0.25, random_state=42)
scaler = StandardScaler()
scaler.fit_transform(X_train)
scaler.transform(X_test)

array([[ 0.6839137 , -0.70579433, -0.625833  , ...,  0.26501306,
        -0.11390738,  0.87654579],
       [-0.52726014, -0.26972894,  0.29889263, ...,  0.48823955,
        -0.94569142, -1.03690611],
       [-0.52726014, -0.39431905, -0.29945925, ..., -0.15517797,
        -0.91619553, -1.03690611],
       ...,
       [ 0.6839137 ,  0.07289387,  0.13570575, ...,  0.21248918,
         0.77981801,  0.09377001],
       [-0.22446668, -1.32874488, -1.06099801, ..., -0.12891603,
        -0.65073254, -0.60203068],
       [-0.22446668, -0.45661411,  0.13570575, ..., -0.81172646,
        -0.77166568, -0.51505559]])

In [43]:
def build_model_activation(hp):
    model = Sequential()

    for i in range(hp.Int('num_layers',1,3)):
        if i ==0:
            model.add(Dense(
                unit = 32,
                activation = hp.Choice(name = 'activation',values = ['relu','tanh','elu']),
                input_dim =8))
        else:
            model.add(Dense(
                unit = 128,
                activation = hp.Choice(name = 'activation',values = ['relu','tanh','elu']),
                kernel_regularizer = regularizers.L2(0.02)
            ))
    model.add(Dense(
        units = 1,
        activation = 'sigmoid'
    ))


    optimizer = hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop', 'adadelta'])
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [44]:
# def build_model_activation(hp):
#     model = Sequential()
#     input_dim = 8

#     for i in range(hp.Int('num_layers', 1, 3)):  # Define the number of layers
#         if i == 0:
#             model.add(Dense(units=32,
#                             activation=hp.Choice('activation_' + str(i), values=['relu', 'tanh', 'elu']),
#                             input_dim=input_dim))
#         else:
#             model.add(Dense(units=128,
#                             activation=hp.Choice('activation_' + str(i), values=['relu', 'tanh', 'elu']),
#                             kernel_regularizer=regularizers.L2(0.01)))

#     model.add(Dense(units=1, activation='sigmoid'))

#     optimizer = hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop', 'adadelta'])
#     model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

#     return model


In [45]:
tuner = kt.RandomSearch(
    hypermodel=build_model_activation,
    objective='val_accuracy',
    max_trials=5,
    directory ='tuner',
    project_name = 'ActivationTuner'
)

In [46]:
tuner.search(X_train,y_train,epochs = 5,validation_data = (X_test,y_test))

Trial 5 Complete [00h 00m 01s]
val_accuracy: 0.640625

Best val_accuracy So Far: 0.6614583134651184
Total elapsed time: 00h 00m 09s
INFO:tensorflow:Oracle triggered exit


In [47]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 1,
 'activation_0': 'elu',
 'optimizer': 'rmsprop',
 'activation_1': 'tanh'}